<a href="https://colab.research.google.com/github/faroukabdelmagied/House-Prices---Advanced-Regression-Techniques/blob/main/Part_5_Model_Development.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importing Libraries

In [ ]:
## SURE
## Importing pd and np
import pandas as pd
import numpy as np
import plotly.express as px

## Importing sklearn models
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.preprocessing import OneHotEncoder, RobustScaler, StandardScaler
from sklearn.compose import ColumnTransformer
from imblearn.pipeline import make_pipeline, Pipeline


/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning:

The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning:

The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.



## Reading the datasets

In [ ]:
## Reading the orginal Engineered set
dataset = pd.read_csv('X_train_featured.csv')
dataset

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,...,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,LotFrontagenan,MasVnrAreanan,GarageYrBltnan
0,1,F,RL,4.174387,9.041922,Pave,Missing,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,5,5,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,...,854,0,7.444249,1,0,2,1,3,1,Gd,8,Typ,0,Missing,Attchd,5.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,Missing,Missing,Missing,0,2,2008,WD,Normal,12.247694,0,0,0
1,2,A,RL,4.382027,9.169518,Pave,Missing,Reg,Lvl,AllPub,FR2,Gtl,Rare_var,Feedr,Norm,1Fam,1Story,6,8,31,31,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,...,0,0,7.140453,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,31.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,Missing,Missing,Missing,0,5,2007,WD,Normal,12.109011,0,0,0
2,3,F,RL,4.219508,9.328123,Pave,Missing,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,7,6,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,...,866,0,7.487734,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,7.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,Missing,Missing,Missing,0,9,2008,WD,Normal,12.317167,0,0,0
3,4,G,RL,4.094345,9.164296,Pave,Missing,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,91,36,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,...,756,0,7.448334,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,8.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,Missing,Missing,Missing,0,2,2006,WD,Abnorml,11.849398,0,0,0
4,5,F,RL,4.430817,9.565214,Pave,Missing,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,8,8,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,...,1053,0,7.695303,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,8.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,Missing,Missing,Missing,0,12,2008,WD,Normal,12.429216,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,F,RL,4.127134,8.976768,Pave,Missing,Reg,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,6,5,8,7,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Gd,TA,No,Unf,0,Unf,0,953,953,GasA,...,694,0,7.406711,0,0,2,1,3,1,TA,7,Typ,1,TA,Attchd,8.0,RFn,2,460,TA,TA,Y,0,40,0,0,0,0,Missing,Missing,Missing,0,8,2007,WD,Normal,12.072541,0,0,0
1456,1457,A,RL,4.442651,9.486076,Pave,Missing,Reg,Lvl,AllPub,Inside,Gtl,NWAmes,Norm,Norm,1Fam,1Story,6,6,32,22,Gable,CompShg,Plywood,Plywood,Stone,119.0,TA,TA,CBlock,Gd,TA,No,ALQ,790,Rec,163,589,1542,GasA,...,0,0,7.636752,1,0,2,0,3,1,TA,7,Min1,2,TA,Attchd,32.0,Unf,2,500,TA,TA,Y,349,0,0,0,0,0,Missing,MnPrv,Missing,0,2,2010,WD,Normal,12.254863,0,0,0
1457,1458,G,RL,4.189655,9.109636,Pave,Missing,Reg,Lvl,AllPub,Inside,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,9,69,4,Gable,CompShg,CemntBd,CmentBd,None,0.0,Ex,Gd,Rare_var,TA,Gd,No,GLQ,275,Unf,0,877,1152,GasA,...,1152,0,7.757906,0,0,2,0,4,1,Gd,9,Typ,2,Gd,Attchd,69.0,RFn,1,252,TA,TA,Y,0,60,0,0,0,0,Missing,GdPrv,Shed,2500,5,2010,WD,Normal,12.493130,0,0,0
1458,1459,A,RL,4.219508,9.181632,Pave,Missing,Reg,Lvl,

In [ ]:
## Reading the dataset with the selected features:
selected_dataset = pd.read_csv('Selected_Features_set.csv')
selected_dataset = selected_dataset.drop(columns=['MSZoning_RL','Condition1_Norm', 'Foundation_CBlock', 'HeatingQC_TA', 'KitchenQual_TA', 'FireplaceQu_Gd'], axis=1)

## Identifying the Selected Features

In [ ]:
Selected_Features = selected_dataset.columns
Selected_Features

Index(['LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF',
       '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'TotRmsAbvGrd', 'Fireplaces',
       'GarageYrBlt', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF',
       'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MoSold'],
      dtype='object')

In [ ]:
## Creating the new dataset:
X = dataset[Selected_Features]
X
## Creating the y data coloumn:
y = dataset[['SalePrice']]

In [ ]:
## Added Features:
added_features = ['MSZoning', 'LotShape', 'BldgType','ExterQual', 
                  'BsmtQual', 'BsmtExposure', 'BsmtFinType1', 'HeatingQC',
                  'CentralAir', 'GrLivArea', 'BsmtFullBath', 'KitchenQual',
                  'Fireplaces', 'GarageType', 'GarageFinish', 'GarageCars', 'PavedDrive']

In [ ]:
## Adding the added features to th  data set:
for feature in added_features:
  X[feature] = dataset[feature]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 40 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   LotArea        1460 non-null   float64
 1   OverallQual    1460 non-null   int64  
 2   OverallCond    1460 non-null   int64  
 3   YearBuilt      1460 non-null   int64  
 4   YearRemodAdd   1460 non-null   int64  
 5   MasVnrArea     1460 non-null   float64
 6   BsmtFinSF1     1460 non-null   int64  
 7   BsmtFinSF2     1460 non-null   int64  
 8   BsmtUnfSF      1460 non-null   int64  
 9   TotalBsmtSF    1460 non-null   int64  
 10  1stFlrSF       1460 non-null   float64
 11  2ndFlrSF       1460 non-null   int64  
 12  LowQualFinSF   1460 non-null   int64  
 13  TotRmsAbvGrd   1460 non-null   int64  
 14  Fireplaces     1460 non-null   int64  
 15  GarageYrBlt    1460 non-null   float64
 16  GarageArea     1460 non-null   int64  
 17  WoodDeckSF     1460 non-null   int64  
 18  OpenPorc

In [ ]:
X.columns

Index(['LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF',
       '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'TotRmsAbvGrd', 'Fireplaces',
       'GarageYrBlt', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF',
       'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MoSold',
       'MSZoning', 'LotShape', 'BldgType', 'ExterQual', 'BsmtQual',
       'BsmtExposure', 'BsmtFinType1', 'HeatingQC', 'CentralAir', 'GrLivArea',
       'BsmtFullBath', 'KitchenQual', 'GarageType', 'GarageFinish',
       'GarageCars', 'PavedDrive'],
      dtype='object')

In [ ]:
X

,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MoSold,MSZoning,LotShape,BldgType,ExterQual,BsmtQual,BsmtExposure,BsmtFinType1,HeatingQC,CentralAir,GrLivArea,BsmtFullBath,KitchenQual,GarageType,GarageFinish,GarageCars,PavedDrive
0,9.041922,7,5,5,5,196.0,706,0,150,856,6.752270,854,0,8,0,5.0,548,0,61,0,0,0,0,2,RL,Reg,1Fam,Gd,Gd,No,GLQ,Ex,Y,7.444249,1,Gd,Attchd,RFn,2,Y
1,9.169518,6,8,31,31,0.0,978,0,284,1262,7.140453,0,0,6,1,31.0,460,298,0,0,0,0,0,5,RL,Reg,1Fam,TA,Gd,Gd,ALQ,Ex,Y,7.140453,0,TA,Attchd,RFn,2,Y
2,9.328123,7,5,7,6,162.0,486,0,434,920,6.824374,866,0,6,1,7.0,608,0,42,0,0,0,0,9,RL,IR1,1Fam,Gd,Gd,Mn,GLQ,Ex,Y,7.487734,1,Gd,Attchd,RFn,2,Y
3,9.164296,7,5,91,36,0.0,216,0,540,756,6.867974,756,0,7,1,8.0,642,0,35,272,0,0,0,2,RL,IR1,1Fam,TA,TA,No,ALQ,Gd,Y,7.448334,1,Gd,Detchd,Unf,3,Y
4,9.565214,8,5,8,8,350.0,655,0,490,1145,7.043160,1053,0,9,1,8.0,836,192,84,0,0,0,0,12,RL,IR1,1Fam,Gd,Gd,Av,GLQ,Ex,Y,7.695303,1,Gd,Attchd,RFn,3,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,8.976768,6,5,8,7,0.0,0,0,953,953,6.859615,694,0,7,1,8.0,460,0,40,0,0,0,0,8,RL,Reg,1Fam,TA,Gd,No,Unf,Ex,Y,7.406711,0,TA,Attchd,RFn,2,Y
1456,9.486076,6,6,32,22,119.0,790,163,589,1542,7.636752,0,0,7,2,32.0,500,349,0,0,0,0,0,2,RL,Reg,1Fam,TA,Gd,No,ALQ,TA,Y,7.636752,1,TA,Attchd,Unf,2,Y
1457,9.109636,7,9,69,4,0.0,275,0,877,1152,7.080026,1152,0,9,2,69.0,252,0,60,0,0,0,0,5,RL,Reg,1Fam,Ex,TA,No,GLQ,Ex,Y,7.757906,0,Gd,Attchd,RFn,1,Y
1458,9.181632,5,6,60,14,0.0,49,1029,0,1078,6.982863,0,0,5,0,60.0,240,366,0,112,0,0,0,4,RL,Reg,1Fam,TA,TA,Mn,GLQ,Gd,Y,6.982863,1,Gd,Attchd,Unf,1,Y


## Applying One Hot Encoder

In [ ]:
## Instance sklearn OneHotEncoder
encoder = OneHotEncoder(categories='auto',
                        #drop='first',
                        sparse=False,
                        handle_unknown='ignore')

## Select and transform only Categorical Features:

## Selecting Categorical:
categorical_features = X.select_dtypes("object").columns

## Tansforming the categorical Columns:
column_trans = ColumnTransformer(
     [
         ("onehot_categorical", encoder, categorical_features),
     ],
     remainder="passthrough",  # to keep the numerical features
 )

## Getting the X_encoded variable:
X_encoded = column_trans.fit_transform(X)

## Train/Test Split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=0)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(1168, 86)
(292, 86)
(1168, 1)
(292, 1)


## Normalizing the Data

In [ ]:
## Normalizing the data using the Standard Scalar

## Instancing the scaler:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

## Creating the SKLEARN Models

In [ ]:
## Instancing the models:
lr = LinearRegression()
rd = Ridge()
rf = RandomForestRegressor()
gbr = GradientBoostingRegressor()

In [ ]:
# Develop function to cross-validate models

def val_model(X, y, regressor, scor, show):

  scores = cross_val_score(regressor, X, y, scoring=scor)

  if show == True:
    print('Score: {:.2f} (+/- {:.2f})'.format(scores.mean(), scores.std()))

  return scores.mean()

In [ ]:
from sklearn.metrics import SCORERS
SCORERS.keys()

dict_keys(['explained_variance', 'r2', 'max_error', 'neg_median_absolute_error', 'neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'neg_root_mean_squared_error', 'neg_mean_poisson_deviance', 'neg_mean_gamma_deviance', 'accuracy', 'roc_auc', 'roc_auc_ovr', 'roc_auc_ovo', 'roc_auc_ovr_weighted', 'roc_auc_ovo_weighted', 'balanced_accuracy', 'average_precision', 'neg_log_loss', 'neg_brier_score', 'adjusted_rand_score', 'homogeneity_score', 'completeness_score', 'v_measure_score', 'mutual_info_score', 'adjusted_mutual_info_score', 'normalized_mutual_info_score', 'fowlkes_mallows_score', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'recall', 'recall_macro', 'recall_micro', 'recall_samples', 'recall_weighted', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weighted', 'jaccard', 'jaccard_macro', 'jaccard_micro', 'jaccard_samples', 'jaccard_weighted'])

In [ ]:
# Create lists to collect name and scores of the models

model = []
score = []

# Check score of each model and append in lists

for regressor in (lr, rd, rf, gbr):
  model.append(regressor.__class__.__name__)
  score.append(val_model(X_train, y_train, regressor, scor='r2', show=True))

pd.DataFrame(data=score, index=model, columns=['R2'])

Score: -20973444852387404167774208.00 (+/- 41946889704774816925483008.00)
Score: 0.90 (+/- 0.04)


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().

/usr/local/lib/python3.6/dist-packages/s

Score: 0.87 (+/- 0.03)


/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/_gb.py:1454: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/_gb.py:1454: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/_gb.py:1454: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/_gb.py:1454: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/_gb.py:1454: DataConversionWarning:


Score: 0.89 (+/- 0.03)


,R2
LinearRegression,-2.097344e+25
Ridge,8.998213e-01
RandomForestRegressor,8.690323e-01
GradientBoostingRegressor,8.866742e-01


## Creating the Pipeline

In [ ]:
## Instancing the best model to be used:
rd = Ridge()

In [ ]:
## Creating the pipeline
pipe = Pipeline(steps=[('encoder', column_trans), ('scaler', scaler), ('Ridge', rd)])

In [ ]:
## Fitting the pipline to the dataset

In [ ]:
pipe.fit(X, y)

Pipeline(memory=None,
         steps=[('encoder',
                 ColumnTransformer(n_jobs=None, remainder='passthrough',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('onehot_categorical',
                                                  OneHotEncoder(categories='auto',
                                                                drop=None,
                                                                dtype=<class 'numpy.float64'>,
                                                                handle_unknown='ignore',
                                                                sparse=False),
                                                  Index(['MSZoning', 'LotShape', 'BldgType', 'ExterQual', 'BsmtQual',
       'BsmtExposure', 'BsmtFinType1', 'HeatingQC', 'CentralAir',
       'KitchenQual', 'GarageType', 'GarageFinish', 'PavedDrive'],
      dtype='ob

## Applying the pipeline on the Test Data

In [ ]:
## Loading the Data:
X_test_df = pd.read_csv('X_test_featured.csv')

In [ ]:
X_test_df

,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MoSold,MSZoning,LotShape,BldgType,ExterQual,BsmtQual,BsmtExposure,BsmtFinType1,HeatingQC,CentralAir,GrLivArea,BsmtFullBath,KitchenQual,GarageType,GarageFinish,GarageCars,PavedDrive
0,9.360655,5,6,49,49,0.0,468.0,144.0,270.0,882.0,6.797940,0,0,5,0,49.0,730.0,140,0,0,0,120,0,6,RH,Reg,1Fam,TA,TA,No,Rec,TA,Y,6.797940,0.0,TA,Attchd,Unf,1.0,Y
1,9.565704,6,6,52,52,108.0,923.0,0.0,406.0,1329.0,7.192182,0,0,6,0,52.0,312.0,393,36,0,0,0,0,6,RL,IR1,1Fam,TA,TA,No,ALQ,TA,Y,7.192182,0.0,Gd,Attchd,Unf,1.0,Y
2,9.534595,5,5,13,12,0.0,791.0,0.0,137.0,928.0,6.833032,701,0,6,1,13.0,482.0,212,34,0,0,0,0,3,RL,IR1,1Fam,TA,Gd,No,GLQ,Gd,Y,7.395722,0.0,TA,Attchd,Fin,2.0,Y
3,9.208138,6,6,12,12,20.0,602.0,0.0,324.0,926.0,6.830874,678,0,7,1,12.0,470.0,360,36,0,0,0,0,6,RL,IR1,1Fam,TA,TA,No,GLQ,Ex,Y,7.380256,0.0,Gd,Attchd,Fin,2.0,Y
4,8.518193,8,5,18,18,0.0,263.0,0.0,1017.0,1280.0,7.154615,0,0,5,0,18.0,506.0,0,82,0,0,144,0,1,RL,IR1,TwnhsE,Gd,Gd,No,ALQ,Ex,Y,7.154615,0.0,Gd,Attchd,RFn,2.0,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,7.568379,4,7,36,36,0.0,0.0,0.0,546.0,546.0,6.302619,546,0,5,0,27.0,0.0,0,0,0,0,0,0,6,RM,Reg,Twnhs,TA,TA,No,Unf,Gd,Y,6.995766,0.0,TA,Missing,Missing,0.0,Y
1455,7.546446,4,5,36,36,0.0,252.0,0.0,294.0,546.0,6.302619,546,0,6,0,36.0,286.0,0,24,0,0,0,0,4,RM,Reg,TwnhsE,TA,TA,No,Rec,TA,Y,6.995766,0.0,TA,CarPort,Unf,1.0,Y
1456,9.903488,5,7,46,10,0.0,1224.0,0.0,0.0,1224.0,7.109879,0,0,7,1,46.0,576.0,474,0,0,0,0,0,9,RL,Reg,1Fam,TA,TA,No,ALQ,Ex,Y,7.109879,1.0,TA,Detchd,Unf,2.0,Y
1457,9.253496,5,5,14,14,0.0,337.0,0.0,575.0,912.0,6.877296,0,0,6,0,27.0,0.0,80,32,0,0,0,0,7,RL,Reg,1Fam,TA,Gd,Av,GLQ,TA,Y,6.877296,0.0,TA,Missing,Missing,0.0,Y


In [ ]:
## Predicting the results using the pipeline:
y_pred = pipe.predict(X_test_df)
pd.DataFrame(y_pred, columns= ['SalePrice'])

,SalePrice
0,11.679270
1,11.911611
2,12.070440
3,12.170524
4,12.142155
...,...
1454,11.353792
1455,11.369388
1456,12.076261
1457,11.703501
